In [1]:
# Run this cell only once
import sys
import os
os.chdir("../..")
print(os.getcwd())
sys.path.insert(1, './code')

/aloy/home/acomajuncosa/PocketVec_v2/GitLab_repo


In [4]:
from utils.pocketvec_utils import prepare_pdb
from utils.pocketvec_utils import select_first_model
from utils.pocketvec_utils import select_chain
from utils.pocketvec_utils import remove_ligands
from utils.pocketvec_utils import remove_hydrogens
from utils.pocketvec_utils import select_occupancies
from utils.pocketvec_utils import extract_ligand_coords
from utils.pocketvec_utils import calculate_centroid
from utils.pocketvec_utils import create_pocket_centroid
from utils.pocketvec_utils import create_fpocket_ds
from utils.pocketvec_utils import read_fpocket_centers
from utils.pocketvec_utils import read_fpocket_scores
from utils.pocketvec_utils import read_prank_scores
import urllib.request
from Bio.PDB import *
import pandas as pd
import numpy as np
import pickle
import shutil
import pybel
import sys
import os

## This notebook exemplifies how to **download**, **process** and **prepare** protein structures and pockets before feeding them into the PocketVec pipeline to generate binding site descriptors. The notebook is divided in 3 sections:

In [5]:
outpath = "examples/1_preparation/"

# PDB CODE
pdb_code = '1A42'

### 1. Protein structure

##### DOWNLOAD #####

In [4]:
# Download pdb file
download = urllib.request.urlretrieve('http://files.rcsb.org/download/' + pdb_code + '.pdb', os.path.join(outpath, pdb_code + ".pdb"))

##### PROCESS #####

In [5]:
# 1. Select the first model
path_in = os.path.join(outpath, pdb_code + ".pdb")
path_out = os.path.join(outpath, pdb_code + "_model.pdb")
select_first_model(path_in, path_out)

In [6]:
# 2. Select the specified chain
chain = 'A'
path_in = path_out
path_out = os.path.join(outpath, pdb_code + "_" + chain + ".pdb")
select_chain(path_in, path_out, chain)

In [7]:
# 3. Remove ligands and waters
path_in = path_out
path_out = os.path.join(outpath, pdb_code + "_ligands.pdb")
ligands = ['HOH', 'BZU', 'HG', 'ZN']
remove_ligands(path_in, path_out, ligands)

In [8]:
# For further processing, we recommend to use the structure_chaking utility from BioBB. You can
# download it from https://github.com/bioexcel/biobb_structure_checking. Change the following
# variable according to your local path 
path_to_check_structure = '/aloy/home/acomajuncosa/programs/structureChecking/bin/check_structure'

# 4. Remove hydrogens
path_in = path_out
path_out = os.path.join(outpath, pdb_code + "_hydrogens.pdb")
logs1 = remove_hydrogens(path_to_check_structure, path_in, path_out)

# 5. Select occupancies
path_in = path_out
path_out = os.path.join(outpath, pdb_code + "_alt.pdb")
logs2 = select_occupancies(path_to_check_structure, path_in, path_out)

##### PREPARE (Should take no more than ~1min for normal sized pdbs)

In [9]:
infile = os.path.join(outpath, pdb_code + "_alt.pdb")
outfile = os.path.join(outpath, pdb_code + "_prepared.mol2")
logfile = os.path.join(outpath, "prep_logs.log")

prepare_pdb(infile, outfile, logfile)

Preparation completed!

### 2. Pocket centroid (ligand-based)

In [10]:
# 1. Extract ligand coordinates
path_to_st = "examples/1_preparation/" + pdb_code + ".pdb"
ligand = 'BZU'
chain = 'A'
coords = extract_ligand_coords(path_to_st, ligand, chain)

In [11]:
# 2. Calculate pocket centroid
centroid = calculate_centroid(coords)

In [12]:
# 3. Create pocket centroid
path_out = os.path.join(outpath, "CTR_LIG.sd")
create_pocket_centroid(centroid, path_out)

1 molecule converted


### 3. Prepare pocket centroid (pocket-detection)

In [4]:
outpath = "examples/1_preparation/"

# To predict pocket locations, we recommend the combined use of fpocket detection and
# prank scoring. You can download the software from https://github.com/Discngine/fpocket
# and https://github.com/rdk/p2rank. Change the following variables according to your local paths

path_to_fpocket = "/aloy/home/acomajuncosa/programs/fpocket/bin/fpocket"
path_to_prank = "/aloy/home/acomajuncosa/programs/p2rank-2.4/distro/prank"

In [12]:
# 1. Change format from mol2 to pdb
path_to_mol2 = os.path.join(outpath, pdb_code + "_prepared.mol2")
path_to_pdb = os.path.join(outpath, pdb_code + "_prepared.pdb")

command = " ".join(['obabel', path_to_mol2, '-O', path_to_pdb])
os.system(command)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1A42_alt.A)

1 molecule converted


0

In [13]:
# 2. Change working directory and run fpocket
os.chdir(outpath)
command = " ".join([path_to_fpocket, '-f', pdb_code + "_prepared.pdb"])
out = os.system(command)
os.chdir("../..")

***** POCKET HUNTING BEGINS ***** 


mkdir: cannot create directory ‘1A42_prepared_out/pockets’: File exists


***** POCKET HUNTING ENDS ***** 


In [14]:
# 3. Rerank with PRANK
create_fpocket_ds(pdb_code)
command = " ".join([path_to_prank, 'rescore', "examples/1_preparation/fpocket3.ds", '-o', 'examples/1_preparation/prank'])
os.system(command)

----------------------------------------------------------------------------------------------
 P2Rank 2.4-beta.2
----------------------------------------------------------------------------------------------

rescoring pockets on proteins from dataset [fpocket3.ds]
processing [1A42_prepared_out.pdb] (1/1)
rescoring finished in 0 hours 0 minutes 4.210 seconds
results saved to directory [/aloy/home/acomajuncosa/PocketVec_v2/GitLab_repo/examples/1_preparation/prank]

----------------------------------------------------------------------------------------------
 finished successfully in 0 hours 0 minutes 5.125 seconds
----------------------------------------------------------------------------------------------


0

In [6]:
# 4. Read fpocket & prank results & calculate pocket centroids
fpockets = read_fpocket_scores(os.path.join(outpath, pdb_code + "_prepared_out", pdb_code + "_prepared_info.txt"))
centers = read_fpocket_centers(os.path.join(outpath, pdb_code + "_prepared_out", pdb_code + "_prepared_pockets.pqr"))
centers = {i: calculate_centroid(centers[i]) for i in sorted(centers)}
pranks = read_prank_scores(os.path.join(outpath, 'prank', pdb_code + "_prepared_out.pdb_rescored.csv"))

In [7]:
# 5. In dicts fpockets (and pranks) we have pocket scores (re-scores) provided by fpocket (prank)
print("FPOCKET Druggability Scores")
print("\n".join([str(i) + " --> " + str(fpockets[i]['Druggability Score']) for i in sorted(fpockets)]))
print("\n\n")
print("PRANK Re-scores")
print("\n".join([str(i) + " --> " + str(pranks[i]) for i in sorted(fpockets)]))

FPOCKET Druggability Scores
1 --> 0.649
2 --> 0.337
3 --> 0.0
4 --> 0.025
5 --> 0.001
6 --> 0.009
7 --> 0.002
8 --> 0.046
9 --> 0.013
10 --> 0.001
11 --> 0.0
12 --> 0.001
13 --> 0.0
14 --> 0.001
15 --> 0.0
16 --> 0.0



PRANK Re-scores
1 --> 12.6436
2 --> 30.1467
3 --> 1.3809
4 --> 1.0933
5 --> 28.399
6 --> 2.3902
7 --> 0.6167
8 --> 9.5715
9 --> 3.2372
10 --> 2.1658
11 --> 2.7282
12 --> 1.3517
13 --> 1.0688
14 --> 0.8161
15 --> 0.9472
16 --> 0.7296


In [21]:
# 6. Print pocket centroids (pocket detection, PD)

if os.path.exists(os.path.join(outpath, "pockets_PD")) == False: os.makedirs(os.path.join(outpath, "pockets_PD"))

for pocket in centers:
    centroid = centers[pocket]
    outfile = os.path.join(outpath, "pockets_PD", "CTR_PD_" + str(pocket) + ".sd")
    create_pocket_centroid(centroid, outfile)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
